In [ ]:

import os
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display
import requests
import gradio as gr

In [ ]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GEMINI_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')

In [ ]:
# constants
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
ollama_url = "http://localhost:11434/v1"
groq_url = "https://api.groq.com/openai/v1"
openrouter_url = "https://openrouter.ai/api/v1"

In [ ]:
# Connect to OpenAI client library
openai = OpenAI()
gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)
ollama = OpenAI(api_key="ollama", base_url=ollama_url)
groq = OpenAI(api_key=groq_api_key, base_url=groq_url)
openrouter = OpenAI(base_url=openrouter_url, api_key=openrouter_api_key)

In [ ]:
system_message = "you are a helpful assistant"

def message_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
    ]
    response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)

    return response.choices[0].message.content

In [ ]:
message_gpt("what is today's date?")

In [ ]:
def printData(data):
    print(f"printing the value {data}")
    return data.upper()

In [ ]:
print(printData("hello"))


In [ ]:
gr.Interface(fn=printData, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True, auth=("riju", "pass123"))

In [ ]:
message_input = gr.Textbox(label="Your message:", info="Enter a message for GPT-4.1-mini", lines=7)
message_output = gr.Textbox(label="Response:", lines=7)

view = gr.Interface(
    fn=message_gpt,
    title="GPT-4.1",
    inputs=[message_input],
    outputs=[message_output],
    examples=["hello", "howdy"],
    flagging_mode="never"
)

view.launch()

In [ ]:
system_message = "You are a helpful assistant who responds in Markdown without code blocks"

message_input = gr.Textbox(label="Your message:", info="Enter a message for GPT-4.1-mini", lines=7)
message_output = gr.Markdown(label="Response:")


view = gr.Interface(
    fn=message_gpt,
    title="GPT-4.1",
    inputs=[message_input],
    outputs=[message_output],
    examples=[
        "Explain the transformer architecture to a layperson", 
        "Explain the transformer architecture to an aspriting AI person"
        ],
    flagging_mode="never"
)

view.launch()

In [ ]:
def chat(message, history):
    return f"history {history}"


In [ ]:
gr.ChatInterface(fn=chat).launch()

In [ ]:
def updated_chat_with_llms(message, history):
    print("asking llm...")
    history=[ {"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": "you are a highly useful assistant"}] + history + [{"role": "user", "content": message}]

    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=messages
    )

    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(fn=updated_chat_with_llms).launch()

In [ ]:
def chat_as_stream(message, history):
    print(f"incoming message: {message}. asking the llm...")
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": "You are a very helpful assistant"}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages, stream=True)
    response = ""

    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
gr.ChatInterface(fn=chat_as_stream).launch()

In [ ]:
shop_system_prompt = """
    You are a helpful assistant who is in charge of a store. Name of the shop is American Eagle.
    Greet and Welcome the customers warmly so they feel valued. 
    Your primary responsibility is to guide the customers who access the chatbot to ask for products, sales and it's availability.
    currently there is a special sales upto 50% discount on suits. There is also a buy 2 get 2 offer running.
    You can encourage the customers to buy more items by prompting the buy 2 get 2 offer, which they might interesting
    Also this is a christmas discount and it runs only till the 31st of december. emphasis on this point without
    being too assertive to highlight the duration of this offer.
"""

In [32]:
def chat_shop(message, history):
    print(f"customer question: {message}")
    final_message = ""
    if "shoes" in message:
        final_message = shop_system_prompt + "if you are looking for shoes, we are out of stock. but we will re-stock it soon. meanwhile present the user with the other options"
    else:
        final_message = shop_system_prompt
        
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": final_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages, stream=True)
    
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [33]:
gr.ChatInterface(fn=chat_shop).launch()

* Running on local URL:  http://127.0.0.1:7872
* To create a public link, set `share=True` in `launch()`.


customer question: hey
customer question: do you have offers on shirts?
customer question: what discounts do you have on shoes?
